# Day 15: Week 3 Review - Accounts & Orders

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class1_fundamentals/week3_accounts_orders/day15_week3_review.ipynb)

---

## Learning Objectives

By the end of this lesson, you will be able to:

1. Review all Week 3 concepts: accounts and order types
2. Choose the right account and order type for any situation
3. Build a simple order management system
4. Demonstrate mastery through a comprehensive quiz

**Time**: 30 min review + 15 min hands-on project

---

# Part 1: Week 3 Review (30 minutes)

---

## Day 11 Review: Brokerage Accounts

### Account Types

| Type | Best For | Key Feature |
|------|----------|-------------|
| **Cash Account** | Beginners, safety | Trade with deposited funds only |
| **Margin Account** | Advanced traders | Borrow up to 50% |
| **IRA/401(k)** | Retirement | Tax advantages |

### Cash vs Margin

```
Cash Account:            Margin Account:
- Trade your money       - Borrow up to 50%
- No margin calls        - Margin call risk
- T+1 settlement wait    - Immediate proceeds
- No short selling       - Can short sell
- Lower risk             - Higher risk/reward
```

### Key Rules

| Rule | Requirement |
|------|-------------|
| **Initial Margin** | 50% (Reg T) |
| **Maintenance Margin** | 25%+ (usually 30-35%) |
| **PDT Rule** | $25K minimum for 4+ day trades/week |
| **SIPC Protection** | $500K (max $250K cash) |

## Day 12 Review: Market and Limit Orders

### Bid-Ask Spread

```
Bid = Highest price BUYERS will pay
Ask = Lowest price SELLERS will accept
Spread = Ask - Bid (your trading cost)
```

### Order Comparison

| Feature | Market Order | Limit Order |
|---------|-------------|-------------|
| **Execution** | Guaranteed | Not guaranteed |
| **Price** | Not guaranteed | Guaranteed or better |
| **Speed** | Immediate | May take time |
| **Best For** | Urgency | Price control |

### Time-in-Force

| Type | Duration |
|------|----------|
| **Day** | Until market close |
| **GTC** | Until filled/cancelled |
| **IOC** | Fill now or cancel |
| **FOK** | Fill ALL now or cancel |

## Day 13 Review: Stop Orders

### Stop Order Types

| Type | Triggers At | Becomes |
|------|-------------|--------|
| **Stop** | Stop price | Market order |
| **Stop-Limit** | Stop price | Limit order |
| **Trailing Stop** | Trail amount from high | Market order |

### Stop Order Risks

```
1. GAP RISK: Price can gap past your stop
   - Stop at $45, opens at $40 = sell at ~$40

2. WHIPSAWS: Tight stops get triggered frequently
   - Normal volatility triggers exit

3. STOP HUNTING: Others may push price to trigger stops
   - Obvious stop levels get targeted
```

### Trailing Stop Behavior

```
Price rises: Stop moves UP (for sells)
Price falls: Stop stays PUT
Price hits stop: Triggers (becomes market)
```

## Day 14 Review: Order Execution

### Order Routing

```
Your Order → Broker → Router → Exchange/Market Maker → Fill
```

### Slippage Sources

| Source | Description |
|--------|-------------|
| **Spread** | Bid-ask difference |
| **Market Impact** | Your order moves price |
| **Timing** | Price changes during routing |

### Minimizing Costs

1. Use **limit orders** when possible
2. Trade **liquid** stocks
3. Trade during **regular hours**
4. **Scale into** large positions

### All Trading Costs

| Cost | Type | Typical |
|------|------|--------|
| Commission | Explicit | $0 |
| Spread | Implicit | $0.01-$0.50 |
| Slippage | Implicit | Variable |
| SEC/FINRA fees | Explicit | <$0.01 |

## Complete Order Type Reference

### Entry Orders

| Scenario | Order Type | Why |
|----------|------------|-----|
| "Buy NOW at any price" | Market | Speed priority |
| "Buy only at $X or less" | Limit | Price control |
| "Buy if breaks above $X" | Stop Buy | Breakout entry |

### Exit Orders

| Scenario | Order Type | Why |
|----------|------------|-----|
| "Sell NOW at any price" | Market | Exit urgency |
| "Sell only at $X or more" | Limit | Target price |
| "Sell if drops below $X" | Stop Sell | Loss protection |
| "Protect my gains" | Trailing Stop | Dynamic exit |

### Advanced Protection

| Scenario | Order Type | Why |
|----------|------------|-----|
| "Sell if drops, but not below $Y" | Stop-Limit | Gap protection |
| "Lock in gains as price rises" | Trailing Stop | Trend following |

## Week 3 Decision Framework

```
OPENING A POSITION:

Do I need to buy/sell RIGHT NOW?
  |
  +-- YES --> Is it a liquid stock?
  |             +-- YES --> Market order OK
  |             +-- NO --> Limit order (at/near ask)
  |
  +-- NO --> Do I have a target price?
               +-- YES --> Limit order at target
               +-- NO --> Limit order near market

---

PROTECTING A POSITION:

What's my priority?
  |
  +-- Guaranteed exit --> Stop order
  |
  +-- Price protection --> Stop-limit (wider gap)
  |
  +-- Let profits run --> Trailing stop
```

---

# Part 2: Hands-On Project (15 minutes)

---

In [ ]:
# Setup - Run this cell first!
import sys
if 'google.colab' in sys.modules:
    !pip install yfinance pandas numpy matplotlib -q
    
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

print("Setup complete!")

## Project: Order Management Simulator

Build a simple system that tracks orders and calculates P&L.

In [ ]:
class OrderManager:
    """
    Simple order management system for learning.
    """
    
    def __init__(self, initial_cash=10000):
        self.cash = initial_cash
        self.positions = {}  # {symbol: {'shares': X, 'avg_cost': Y}}
        self.orders = []  # Order history
        self.trades = []  # Executed trades
    
    def get_current_price(self, symbol):
        """Fetch current price from yfinance."""
        try:
            ticker = yf.Ticker(symbol)
            hist = ticker.history(period='1d')
            if not hist.empty:
                return hist['Close'].iloc[-1]
        except:
            pass
        return None
    
    def place_market_order(self, symbol, shares, side='buy'):
        """Execute a market order."""
        price = self.get_current_price(symbol)
        if price is None:
            print(f"Could not get price for {symbol}")
            return False
        
        # Add small slippage simulation (0.05%)
        slippage = price * 0.0005
        if side == 'buy':
            exec_price = price + slippage
            cost = exec_price * shares
            
            if cost > self.cash:
                print(f"Insufficient funds. Need ${cost:.2f}, have ${self.cash:.2f}")
                return False
            
            self.cash -= cost
            
            # Update position
            if symbol in self.positions:
                old_shares = self.positions[symbol]['shares']
                old_cost = self.positions[symbol]['avg_cost']
                new_shares = old_shares + shares
                self.positions[symbol]['avg_cost'] = ((old_shares * old_cost) + (shares * exec_price)) / new_shares
                self.positions[symbol]['shares'] = new_shares
            else:
                self.positions[symbol] = {'shares': shares, 'avg_cost': exec_price}
        
        else:  # sell
            exec_price = price - slippage
            
            if symbol not in self.positions or self.positions[symbol]['shares'] < shares:
                print(f"Insufficient shares to sell")
                return False
            
            proceeds = exec_price * shares
            self.cash += proceeds
            
            self.positions[symbol]['shares'] -= shares
            if self.positions[symbol]['shares'] == 0:
                del self.positions[symbol]
        
        # Record trade
        trade = {
            'time': datetime.now(),
            'symbol': symbol,
            'side': side,
            'shares': shares,
            'price': exec_price,
            'type': 'market'
        }
        self.trades.append(trade)
        
        print(f"FILLED: {side.upper()} {shares} {symbol} @ ${exec_price:.2f}")
        return True
    
    def get_portfolio_value(self):
        """Calculate total portfolio value."""
        total = self.cash
        
        for symbol, pos in self.positions.items():
            price = self.get_current_price(symbol)
            if price:
                total += price * pos['shares']
        
        return total
    
    def show_positions(self):
        """Display current positions."""
        print("\nCurrent Positions")
        print("=" * 60)
        print(f"Cash: ${self.cash:,.2f}")
        print("-" * 60)
        
        if not self.positions:
            print("No stock positions")
        else:
            print(f"{'Symbol':<10} {'Shares':>10} {'Avg Cost':>12} {'Current':>12} {'P&L':>12}")
            for symbol, pos in self.positions.items():
                current = self.get_current_price(symbol) or pos['avg_cost']
                pnl = (current - pos['avg_cost']) * pos['shares']
                pnl_pct = (current / pos['avg_cost'] - 1) * 100
                print(f"{symbol:<10} {pos['shares']:>10} ${pos['avg_cost']:>10.2f} ${current:>10.2f} ${pnl:>10.2f} ({pnl_pct:+.1f}%)")
        
        print("-" * 60)
        print(f"Total Portfolio Value: ${self.get_portfolio_value():,.2f}")
    
    def show_trades(self):
        """Display trade history."""
        print("\nTrade History")
        print("=" * 70)
        if not self.trades:
            print("No trades yet")
        else:
            for trade in self.trades:
                print(f"{trade['time'].strftime('%H:%M:%S')} | {trade['side'].upper():4} {trade['shares']:>5} {trade['symbol']:<6} @ ${trade['price']:.2f} ({trade['type']})")

# Create order manager
om = OrderManager(initial_cash=10000)
print("Order Manager initialized with $10,000")
om.show_positions()

In [ ]:
# Execute some trades
print("\n" + "="*60)
print("EXECUTING TRADES")
print("="*60)

# Buy some stocks
om.place_market_order('AAPL', 10, 'buy')
om.place_market_order('MSFT', 10, 'buy')
om.place_market_order('GOOGL', 5, 'buy')

In [ ]:
# View current positions
om.show_positions()

In [ ]:
# View trade history
om.show_trades()

In [ ]:
# Sell some shares
print("\n" + "="*60)
print("PARTIAL SELL")
print("="*60)

om.place_market_order('AAPL', 5, 'sell')
om.show_positions()

## Exercise: Extend the Order Manager

Try adding these features:
1. Limit orders (check if price meets limit before executing)
2. Stop orders (monitor price and trigger when stop is hit)
3. Position sizing (never risk more than X% per trade)

In [ ]:
# TODO: Add limit order functionality
def place_limit_order(self, symbol, shares, limit_price, side='buy'):
    """
    Place a limit order.
    Only executes if current price is favorable.
    """
    current_price = self.get_current_price(symbol)
    
    if current_price is None:
        print(f"Could not get price for {symbol}")
        return False
    
    if side == 'buy':
        # For buys, current ask must be <= limit
        if current_price <= limit_price:
            print(f"Limit buy triggered at ${current_price:.2f} (limit: ${limit_price:.2f})")
            return self.place_market_order(symbol, shares, 'buy')
        else:
            print(f"Price ${current_price:.2f} above limit ${limit_price:.2f} - order not filled")
            return False
    else:
        # For sells, current bid must be >= limit
        if current_price >= limit_price:
            print(f"Limit sell triggered at ${current_price:.2f} (limit: ${limit_price:.2f})")
            return self.place_market_order(symbol, shares, 'sell')
        else:
            print(f"Price ${current_price:.2f} below limit ${limit_price:.2f} - order not filled")
            return False

# Add method to class
OrderManager.place_limit_order = place_limit_order

# Test limit order
print("\nTesting Limit Order:")
print("="*60)
current_aapl = om.get_current_price('AAPL')
print(f"Current AAPL price: ${current_aapl:.2f}")

# This should NOT fill (limit too low)
om.place_limit_order('AAPL', 5, current_aapl - 10, 'buy')

# This SHOULD fill (limit above market)
om.place_limit_order('AAPL', 5, current_aapl + 5, 'buy')

---

# Part 3: Comprehensive Week 3 Quiz

---

In [ ]:
# Week 3 Comprehensive Quiz

questions = [
    # Day 11: Accounts
    {
        "question": "In a cash account, you can trade with:",
        "options": ["A) Borrowed funds", "B) Only deposited funds",
                   "C) 2x leverage", "D) Unlimited funds"],
        "answer": "B",
        "topic": "Accounts"
    },
    {
        "question": "The PDT rule requires $25K minimum for:",
        "options": ["A) Any trading", "B) Options trading",
                   "C) 4+ day trades in 5 days", "D) Margin trading"],
        "answer": "C",
        "topic": "Accounts"
    },
    {
        "question": "Standard initial margin requirement is:",
        "options": ["A) 25%", "B) 50%", "C) 75%", "D) 100%"],
        "answer": "B",
        "topic": "Accounts"
    },
    # Day 12: Market/Limit Orders
    {
        "question": "The 'ask' price is:",
        "options": ["A) Highest buyers will pay", "B) Lowest sellers will accept",
                   "C) Yesterday's close", "D) Average price"],
        "answer": "B",
        "topic": "Orders"
    },
    {
        "question": "A market order guarantees:",
        "options": ["A) Price", "B) Execution", "C) Both", "D) Neither"],
        "answer": "B",
        "topic": "Orders"
    },
    {
        "question": "A limit order guarantees:",
        "options": ["A) Price or better", "B) Execution", "C) Both", "D) Neither"],
        "answer": "A",
        "topic": "Orders"
    },
    {
        "question": "GTC orders remain active:",
        "options": ["A) Until market close", "B) For one week",
                   "C) Until filled or cancelled", "D) For 30 days max"],
        "answer": "C",
        "topic": "Orders"
    },
    # Day 13: Stop Orders
    {
        "question": "A stop order becomes what when triggered?",
        "options": ["A) Limit order", "B) Market order",
                   "C) Cancelled", "D) GTC order"],
        "answer": "B",
        "topic": "Stops"
    },
    {
        "question": "A stop-limit order becomes what when triggered?",
        "options": ["A) Market order", "B) Limit order",
                   "C) Day order", "D) Stop order"],
        "answer": "B",
        "topic": "Stops"
    },
    {
        "question": "A trailing stop for a long position:",
        "options": ["A) Only moves down", "B) Only moves up",
                   "C) Moves both ways", "D) Never moves"],
        "answer": "B",
        "topic": "Stops"
    },
    # Day 14: Execution
    {
        "question": "Slippage is:",
        "options": ["A) Broker commission", "B) Difference between expected and actual price",
                   "C) Settlement delay", "D) Account fee"],
        "answer": "B",
        "topic": "Execution"
    },
    {
        "question": "To minimize slippage, you should:",
        "options": ["A) Use market orders", "B) Trade illiquid stocks",
                   "C) Trade liquid stocks with limit orders", "D) Trade after hours"],
        "answer": "C",
        "topic": "Execution"
    },
    {
        "question": "US stock settlement is currently:",
        "options": ["A) T+0", "B) T+1", "C) T+2", "D) T+3"],
        "answer": "B",
        "topic": "Execution"
    },
    {
        "question": "PFOF means brokers receive payment from:",
        "options": ["A) You", "B) Exchanges", "C) Market makers", "D) SEC"],
        "answer": "C",
        "topic": "Execution"
    },
    {
        "question": "VWAP stands for:",
        "options": ["A) Variable Weighted Average Price", "B) Volume Weighted Average Price",
                   "C) Value Weighted Average Price", "D) Volatility Weighted Average Price"],
        "answer": "B",
        "topic": "Execution"
    }
]

def run_week3_quiz():
    score = 0
    topic_scores = {}
    
    print("Week 3 Comprehensive Quiz: Accounts & Orders")
    print("=" * 60)
    print(f"Total Questions: {len(questions)}")
    print("Pass: 80% (12/15)")
    print("=" * 60)
    
    for i, q in enumerate(questions, 1):
        print(f"\nQ{i} ({q['topic']}): {q['question']}")
        for opt in q['options']:
            print(f"   {opt}")
        
        answer = input("Your answer (A/B/C/D): ").strip().upper()
        
        if q['topic'] not in topic_scores:
            topic_scores[q['topic']] = {'correct': 0, 'total': 0}
        topic_scores[q['topic']]['total'] += 1
        
        if answer == q['answer']:
            print("Correct!")
            score += 1
            topic_scores[q['topic']]['correct'] += 1
        else:
            print(f"Incorrect. The answer is {q['answer']}")
    
    print(f"\n{'='*60}")
    print(f"Final Score: {score}/{len(questions)} ({score/len(questions)*100:.0f}%)")
    print("\nScores by Topic:")
    for topic, scores in topic_scores.items():
        pct = scores['correct']/scores['total']*100
        status = "PASS" if pct >= 80 else "REVIEW"
        print(f"  {topic}: {scores['correct']}/{scores['total']} ({pct:.0f}%) - {status}")
    
    if score >= 12:
        print("\n*** PASSED! You've mastered Week 3. Ready for Week 4! ***")
    else:
        print("\nPlease review the topics marked 'REVIEW' before moving to Week 4.")

# Uncomment to run quiz
# run_week3_quiz()

---

## Week 3 Summary

### What You Learned

| Day | Topic | Key Skill |
|-----|-------|----------|
| 11 | Brokerage Accounts | Choose right account type |
| 12 | Market & Limit Orders | Basic order placement |
| 13 | Stop Orders | Protect positions |
| 14 | Order Execution | Minimize trading costs |
| 15 | Review | Apply all concepts |

### Action Items Before Week 4

1. **If no account yet**: Open a paper trading account
2. **Practice**: Place each order type in paper trading
3. **Know your broker**: Review their execution quality reports
4. **Set rules**: Determine your default order types

### Next Week Preview: Charts & Psychology

- Day 16: Candlestick Charts
- Day 17: Volume Analysis
- Day 18: Timeframes
- Day 19: Trading Psychology
- Day 20: Class 1 Final Review

---

*Money Talks - Trading & Investing Education*